In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/moai-2025-training/test_images.pt
/kaggle/input/moai-2025-training/train_images.pt
/kaggle/input/moai-2025-training/train_labels.csv


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from torchvision import models, transforms
import pandas as pd
import numpy as np
import warnings
from torch.serialization import SourceChangeWarning

warnings.filterwarnings("ignore", category=SourceChangeWarning)

# 小題1：數據讀取與預處理
# 讀取數據
try:
    # 明确指定weights_only=False
    train_images = torch.load('/kaggle/input/moai-2025-training/train_images.pt', weights_only=False)
    test_images = torch.load('/kaggle/input/moai-2025-training/test_images.pt', weights_only=False)
except Exception as e:
    print(f"加载数据时发生错误: {e}")
    # 可以添加备选加载方案或退出逻辑
    raise

train_labels = pd.read_csv('/kaggle/input/moai-2025-training/train_labels.csv').values[:, 1]  # 假設CSV有索引列

# 調整數據形狀以適應遷移學習模型
train_images = train_images.unsqueeze(1).repeat(1, 3, 1, 1)  # 複製通道維度
test_images = test_images.unsqueeze(1).repeat(1, 3, 1, 1)    # 複製通道維度

# 將數據轉換為浮點型並歸一化
train_images = train_images.float() / 255.0
test_images = test_images.float() / 255.0

# 創建數據集和數據加載器
dataset = TensorDataset(train_images, torch.tensor(train_labels, dtype=torch.long))
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_images, batch_size=64, shuffle=False)

In [9]:
# 小題2：構建CNN模型（使用遷移學習）
class DigitClassifier(nn.Module):
    def __init__(self):
        super(DigitClassifier, self).__init__()
        # 使用預訓練的ResNet18作為特徵提取器
        self.feature_extractor = models.resnet18(pretrained=True)
        
        # 調整輸入層以適應28x28的圖像
        self.feature_extractor.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        
        # 凍結大部分預訓練參數
        for param in list(self.feature_extractor.parameters())[:-5]:
            param.requires_grad = False
        
        # 修改最後一層以適應10類分類任務
        num_ftrs = self.feature_extractor.fc.in_features
        self.feature_extractor.fc = nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        return self.feature_extractor(x)

model = DigitClassifier()

# 小題3：訓練模型
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 訓練循環
num_epochs = 10
for epoch in range(num_epochs):
    # 訓練階段
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    train_acc = 100.0 * correct / total
    
    # 驗證階段
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    val_acc = 100.0 * correct / total
    
    print(f'Epoch {epoch+1}/{num_epochs}')
    print(f'Train Loss: {train_loss/len(train_loader):.4f} | Train Acc: {train_acc:.2f}%')
    print(f'Val Loss: {val_loss/len(val_loader):.4f} | Val Acc: {val_acc:.2f}%')
    print('-' * 50)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10
Train Loss: 0.4732 | Train Acc: 84.89%
Val Loss: 0.2853 | Val Acc: 90.69%
--------------------------------------------------
Epoch 2/10
Train Loss: 0.3081 | Train Acc: 90.06%
Val Loss: 0.2652 | Val Acc: 91.28%
--------------------------------------------------
Epoch 3/10
Train Loss: 0.2591 | Train Acc: 91.56%
Val Loss: 0.2436 | Val Acc: 92.11%
--------------------------------------------------
Epoch 4/10
Train Loss: 0.2258 | Train Acc: 92.62%
Val Loss: 0.2393 | Val Acc: 92.69%
--------------------------------------------------
Epoch 5/10
Train Loss: 0.1945 | Train Acc: 93.61%
Val Loss: 0.2452 | Val Acc: 92.38%
--------------------------------------------------
Epoch 6/10
Train Loss: 0.1779 | Train Acc: 94.22%
Val Loss: 0.2550 | Val Acc: 92.61%
--------------------------------------------------
Epoch 7/10
Train Loss: 0.1559 | Train Acc: 94.90%
Val Loss: 0.2438 | Val Acc: 92.78%
--------------------------------------------------
Epoch 8/10
Train Loss: 0.1368 | Train Acc: 95.60

In [14]:
import torch
import pandas as pd

test_images = torch.load('/kaggle/input/moai-2025-training/test_images.pt', weights_only=True)
test_images = (test_images.float() - test_images.float().mean()) /test_images.float().std()

model.eval()
test_predictions = []

with torch.no_grad():
    try:
        for inputs in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            test_predictions.extend(predicted.cpu().numpy())
    except Exception as e:
        print(f"预测过程中出现错误: {e}")

df_test = pd.DataFrame({"label": test_predictions})
df_test.to_csv("submission.csv", index_label="id")

In [10]:
# 生成測試集預測
model.eval()
test_predictions = []

with torch.no_grad():
    try:
        for inputs in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            test_predictions.extend(predicted.cpu().numpy())
    except Exception as e:
        print(f"预测过程中出现错误: {e}")

# 保存預測結果
submission = pd.DataFrame({'Id': range(len(test_predictions)), 'Category': test_predictions})
submission.to_csv('submission.csv', index=False)